In [1]:
import numpy as np

data_path = 'part-000-00000.npy'
dtype = np.int32
token_ids = np.memmap(data_path, mode="r", dtype=dtype)

# truncate
token_ids = token_ids[:1_000_000]

In [5]:
from transformers import AutoTokenizer
import json

strings = []
with open('arc_c.jsonl') as f:
    for line in f:
        doc = json.loads(line)
        strings.append(doc['request']['context'])
        strings.append(doc['request']['continuation'])

text = '\n'.join(strings)

tokenizer = AutoTokenizer.from_pretrained("allenai/dolma2-tokenizer")
tokens = tokenizer.encode(text)
tokens = np.array(tokens, dtype=np.int32)

print(f"Number of tokens: {len(tokens):,}")

Token indices sequence length is longer than the specified maximum sequence length for this model (921314 > 8192). Running this sequence through the model will result in indexing errors


Number of tokens: 921,314


In [8]:
import gzip
import io

def compress_array_as_bytes(arr: np.ndarray) -> int:
    buffer = io.BytesIO()
    with gzip.GzipFile(fileobj=buffer, mode='wb') as f:
        f.write(arr.tobytes())
    return len(buffer.getvalue())

tokens_size = compress_array_as_bytes(tokens)
token_ids_size = compress_array_as_bytes(token_ids)
concat_size = compress_array_as_bytes(np.concatenate([tokens, token_ids]))

mutual_info = tokens_size + token_ids_size - concat_size
print(f"Mutual information: {mutual_info:,} bytes")

Mutual information: 319 bytes
